<a href="https://colab.research.google.com/github/cagBRT/Machine-Learning/blob/master/Cars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Clone the entire repo.
!git clone -l -s https://github.com/cagBRT/Machine-Learning.git cloned-repo
%cd cloned-repo
!ls

In [0]:
from IPython.display import Image
def page(num):
    #print(str(num))
    return Image("Intro to Machine Learning Programming ("+ str(num) + ").png")
print("done")

In [0]:
from IPython.display import Image
Image("Intro to Machine Learning Programming.png")
print("done")

In [0]:
# Use seaborn for pairplot
!pip install seaborn


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)


#Regression: predict fuel efficiency




### Get the data
First download the dataset.

In [0]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
print("done")

In [0]:
dataset.head()

### Prepare the data

###Clean the data

In [0]:
dataset.isna().sum()

For this tutorial, drop the rows that contain NA. For some cases you make need to change missing values instead.

In [0]:
dataset = dataset.dropna()
dataset.isna().sum()

In [0]:
dataset.describe()

You could also look at the number of times each value appears in a column. 

In [0]:
dataset['Cylinders'].value_counts()

###Convert categorical data 

In [0]:
origin = dataset.pop('Origin')
print("done")

In [0]:
dataset.tail()

In [0]:
dataset['USA'] = (origin == 1)*1.0
dataset['Europe'] = (origin == 2)*1.0
dataset['Japan'] = (origin == 3)*1.0
dataset.tail()

###Split the data into train and test sets
Now split the dataset into a training set and a test set.

We will use the test set in the final evaluation of our model.

In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
print("done")

### Inspect the data

Have a quick look at the joint distribution of a few pairs of columns from the training set.

In [0]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")
plt.show()

Have a quick look at the joint distribution of a few pairs of columns from the test set.

In [0]:
sns.pairplot(test_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")
plt.show()

Also look at the overall statistics.<br> The stats for the training set and the test set should be similiar.

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

In [0]:
test_stats = test_dataset.describe()
test_stats.pop("MPG")
test_stats = test_stats.transpose()
test_stats

### Split features from labels

Separate the target value, or "label", from the features. This label is the value that you will train the model to predict.

In [0]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')
print("done")

### Normalize the data

Look again at the `train_stats` block above and note how different the ranges of each feature are.

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)
print("done")

## The model

###Define the model

In [0]:
from keras import metrics

inputs = len(train_dataset.keys())
print("number of inputs to the model = " + str(inputs))

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation=tf.nn.relu,input_shape=([len(train_dataset.keys())]),),
    layers.Dense(64, activation=tf.nn.relu),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mean_squared_error', 
                optimizer=optimizer,
                metrics=['mean_absolute_error', 'mean_squared_error'])
  return model
  print("done")

### Build the model

Let's build our model. Here, we'll use a `Sequential` model with two densely connected hidden layers, and an output layer that returns a single, continuous value. The model building steps are wrapped in a function, `build_model`, since we'll create a second model, later on.

In [0]:
model = build_model()
print("done")

In [0]:
model.summary()

### Train the model

Train the model for 1000 epochs, and record the training and validation accuracy in the `history` object.

ML Vocabulary: https://towardsdatascience.com/epoch-vs-iterations-vs-batch-size-4dfb9c7ce9c9

In [0]:
# Display training progress by printing a single dot for each completed epoch
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 100

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])

In [0]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [MPG]')
  plt.plot(hist['epoch'], hist['mean_absolute_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_absolute_error'],
           label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$MPG^2$]')
  plt.plot(hist['epoch'], hist['mean_squared_error'],
           label='Train Error')
  plt.plot(hist['epoch'], hist['val_mean_squared_error'],
           label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

Visualize the model's training progress using the stats stored in the `history` object.

In [0]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(normed_train_data, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[early_stop, PrintDot()])

plot_history(history)

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=1)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))


### Test the model

Finally, predict MPG values using data in the testing set:

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
plt.axis('equal')
plt.axis('square')
plt.xlim([0,plt.xlim()[1]])
plt.ylim([0,plt.ylim()[1]])
_ = plt.plot([-100, 100], [-100, 100])
plt.show()

In [0]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")
plt.show()

In [0]:
#Normalized data
trial=([1, 1, 1, 1, 1, 1, 1.0, 0.0, 0.0],)
trial_predictions = model.predict(trial).flatten()
print("done")

In [0]:
print(trial_predictions)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()
lr.fit(train_dataset, train_labels)
y_pred = lr.predict(test_dataset)

print(mean_absolute_error(test_labels, y_pred))

Add the missing data:

#1971 Ford Pinto
dataset.at[32,"Horsepower"] = 100
print(dataset.loc[32])

#1974 Ford Maverick
dataset.at[126,"Horsepower"] = 75
print(dataset.loc[126])

#1980 Renault Le Car De Luxe
dataset.at[330,"Horsepower"] = 50
print(dataset.loc[330])

#1980 Ford Mustang Cobra
dataset.at[336,"Horsepower"] = 88
print(dataset.loc[336])

#1981 Renault 18i
dataset.at[354,"Horsepower"] = 81
print(dataset.loc[354])
#1982 AMC Concord DL
dataset.at[374,"Horsepower"] = 82
print(dataset.loc[374])